**1. Importing the librairies**

In [1]:
import pandas as pd
# Importing Models
import warnings
warnings.simplefilter('ignore')

In [2]:
# standard imports and setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
# model evaluation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit, cross_validate, StratifiedKFold
from sklearn.metrics import *
# pipelines
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
# data preparation
from sklearn.preprocessing import *
# from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, RFECV
from sklearn.utils import resample
# from imblearn.datasets import make_imbalance
# from imblearn.over_sampling import SMOTE
# # machine learning
# from sklearn.linear_model import *
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC, LinearSVC
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier, XGBRegressor
#------------------
from __future__ import division
from __future__ import print_function
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from sklearn.metrics import roc_curve, roc_auc_score
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from time import time
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.metrics import plot_precision_recall_curve
from pyod.models.hbos import HBOS


In [3]:
 
import pandas as pd
dataset = pd.read_csv(r"C:\Users\NaumanRafique\Downloads\nourasaeed721-attachments\wustl.csv",usecols=['SIntPkt','Pulse_Rate',
 'dMaxPktSz','DstLoad', 'Trans','SrcLoad','sMinPktSz','DIntPktAct','DIntPkt','DstBytes','SpO2','DstGap','Loss','Label','DIA',
 'TotPkts','sMaxPktSz','Load','Rate','pSrcLoss','SrcBytes','DstJitter','Temp','SYS','pDstLoss','Heart_rate','dMinPktSz', 'SrcJitter','SIntPktAct','Resp_Rate','TotBytes','ST','pLoss','Dur','SrcGap'])

print(dataset.shape)
 

(16318, 35)


In [4]:
2+2

4

In [5]:
df_columns = ['Data Sample', 'Testing Size',  
              'ABOD', 'CBLOF', 'FB',
              'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM'
              ,'PCA'
             ]
 







random_state = 42




from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

import pandas as pd

# creating data frames for calculating 

df_time = pd.DataFrame(columns=df_columns)
df_precision = pd.DataFrame(columns=df_columns)
df_recall = pd.DataFrame(columns=df_columns)

# List of time,precision,recall 

list_time  =[]
list_precision  =[]
list_recall  =[]

dataset_sample = [1000,5000,10000,15000,16318]
testing_size= [0.2,0.3,0.4]

loc = 0

for sample in dataset_sample:
    
    dataset_sample = dataset.sample(sample)
    
    x = dataset_sample.iloc[:,:-1].values
    y = dataset_sample.iloc[:,-1].values
        
        
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
        
        
    classifierslist = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
    }

        
        
    
    for size in testing_size:
        
        
        
        x_train, x_test, y_train, y_test= train_test_split(x,y, test_size = size, random_state =0)
        
        x_train = sc.fit_transform(x_train)
        x_test = sc.fit_transform(x_test)  
        
      
        
        
        
        
        list_time.extend([sample,size])
        list_precision.extend([sample,size])
        list_recall.extend([sample,size])
        
        for clf_name, clf in classifierslist.items():
            
            t0 = time()
            clf.fit(x_train)
            test_scores = clf.decision_function(x_test)
            t1 = time()
            duration = round(t1 - t0, ndigits=4)
            
            roc = round(roc_auc_score(y_test, test_scores), ndigits=4)            
            prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
             
            
            list_time.append(duration)
            list_precision.append(prn)
            list_recall.append(roc)
        df_time.loc[loc] =list_time
        df_precision.loc[loc] =list_precision
        df_recall.loc[loc] =list_recall
        list_time  =[]
        list_precision  =[]
        list_recall  =[]
        loc+=1
        
            

            #accuracy = metrics.accuracy_score(y_test, y_pred)
#             y_test_pred = clf.predict(x_test)  # outlier labels (0 or 1)

#             print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
#                   'execution time: {duration}s'.format(clf_name=clf_name, roc=roc, prn=prn, duration=duration))

#             roc_list.append(roc)
#             prn_list.append(prn)

        






In [6]:
df_time

,Data Sample,Testing Size,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,1000.0,0.2,3.3494,2.4719,0.1879,2.6048,0.4088,0.0589,0.0180,1.3179,0.1139,0.0085
1,1000.0,0.3,0.2928,0.1179,0.2708,0.0220,0.4588,0.0809,0.0260,1.3108,0.1229,0.0100
2,1000.0,0.4,0.3058,0.0949,0.2958,0.0240,0.4308,0.1049,0.0260,1.6116,0.0800,0.0080
3,5000.0,0.2,1.5061,0.2199,1.7930,0.0340,0.8105,0.4837,0.1689,2.4795,3.0593,0.0221
4,5000.0,0.3,1.5011,0.1549,1.3832,0.0280,1.0804,0.5137,0.1389,1.8853,2.4766,0.0219
5,5000.0,0.4,1.5021,0.1729,1.2303,0.0310,0.7416,0.5577,0.1079,1.8247,2.0498,0.0220
6,10000.0,0.2,3.3101,0.3198,3.6609,0.0510,1.6141,1.5141,0.3428,2.7680,12.4778,0.0429
7,10000.0,0.3,3.2701,0.2229,3.4040,0.0560,1.6620,2.1248,0.4547,2.8389,13.5882,0.0478
8,10000.0,0.4,4.5094,0.2539,3.9147,0.0510,2.7104,5.9366,0.6426,3.2661,9.3556,0.0540
9,15000.0,0.2,8.4102,0.5037,10.2221,0.0840,4.8552,4.8382,0.9744,4.2159,31.9506,0.0740


In [7]:
df_recall

,Data Sample,Testing Size,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,1000.0,0.2,0.6488,0.6669,0.5760,0.6651,0.6417,0.6850,0.5586,0.6313,0.6386,0.6485
1,1000.0,0.3,0.6256,0.6691,0.5282,0.6600,0.6373,0.6976,0.5057,0.6101,0.6411,0.6518
2,1000.0,0.4,0.5967,0.6289,0.5281,0.6438,0.6080,0.6518,0.5107,0.5915,0.5975,0.6124
3,5000.0,0.2,0.5992,0.6350,0.4825,0.5612,0.6430,0.6791,0.4821,0.6154,0.6387,0.6472
4,5000.0,0.3,0.6083,0.6470,0.4885,0.5600,0.6558,0.6780,0.4859,0.6073,0.6459,0.6554
5,5000.0,0.4,0.6385,0.6544,0.5171,0.5674,0.6941,0.6805,0.5155,0.6386,0.6647,0.6743
6,10000.0,0.2,0.6155,0.6685,0.5417,0.5912,0.6503,0.6622,0.5432,0.6192,0.6536,0.6617
7,10000.0,0.3,0.5942,0.6249,0.5083,0.7116,0.7166,0.6812,0.5170,0.6270,0.6681,0.6769
8,10000.0,0.4,0.6206,0.6401,0.5149,0.7171,0.6999,0.6924,0.5172,0.6363,0.6747,0.6839
9,15000.0,0.2,0.5791,0.6305,0.5133,0.6876,0.7182,0.6937,0.5186,0.6349,0.6614,0.6730


In [8]:
df_precision

,Data Sample,Testing Size,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,1000.0,0.2,0.3846,0.4615,0.3077,0.4615,0.4615,0.4231,0.2692,0.4615,0.4231,0.4231
1,1000.0,0.3,0.3077,0.4359,0.0513,0.4359,0.4359,0.4359,0.0256,0.4103,0.4359,0.4359
2,1000.0,0.4,0.3396,0.3774,0.0943,0.4151,0.3774,0.3962,0.0377,0.3585,0.3774,0.3774
3,5000.0,0.2,0.2419,0.2903,0.0645,0.1870,0.4677,0.3871,0.0403,0.4516,0.4677,0.4758
4,5000.0,0.3,0.1761,0.2727,0.0682,0.1648,0.4716,0.4034,0.0568,0.4545,0.4716,0.4773
5,5000.0,0.4,0.2532,0.2704,0.0601,0.1545,0.5193,0.3820,0.0601,0.4979,0.5064,0.5193
6,10000.0,0.2,0.1181,0.4134,0.0669,0.2126,0.4803,0.4764,0.0669,0.4685,0.4921,0.4882
7,10000.0,0.3,0.1440,0.1840,0.0853,0.3227,0.4853,0.4853,0.0827,0.4693,0.3840,0.4533
8,10000.0,0.4,0.1755,0.2189,0.0907,0.3649,0.4931,0.4556,0.0828,0.4773,0.4063,0.4753
9,15000.0,0.2,0.1077,0.2231,0.1179,0.3205,0.4872,0.4410,0.1077,0.4692,0.3667,0.4051


In [9]:

df_time.to_csv("TimeData.csv")
df_precision.to_csv("PrecisionData.csv")
df_recall.to_csv("RecallData.csv")

Making confusion matrix and measurment the accur